In [125]:
import requests
import pandas as pd
import numpy as np
import io
from datetime import datetime, timedelta
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Constants
API_KEY = '2lZRFGaqFiEYkzr7WUuT4EaoC1X'  # Replace with your actual API key
SINCE_DATE = int(datetime(2024, 1, 17).timestamp())  # Jan 1, 2015
UNTIL_DATE = int(datetime.now().timestamp())  # Current date

# URLs for fetching data
PRICE_URL = 'https://api.glassnode.com/v1/metrics/market/price_usd_close'
METRICS = [
    'https://api.glassnode.com/v1/metrics/institutions/us_spot_etf_flows_net'
]

# Dictionary for stacked metrics: URL -> key to extract
METRICS_STACKED = {
    'https://api.glassnode.com/v1/metrics/institutions/us_spot_etf_balances_all': ['total','GBTC'],
    'https://api.glassnode.com/v1/metrics/institutions/us_spot_etf_flows_all': ['total','GBTC']
}

def fetch_glassnode_data(url, asset='BTC', key_to_extract=None):
    params = {
        'a': asset,
        's': SINCE_DATE,
        'u': UNTIL_DATE,
        'api_key': API_KEY,
        'c': 'USD'
    }
    
    metric_base_name = url.split('/')[-1]
    
    # Set format based on whether it's a stacked metric
    if url in METRICS_STACKED:
        params['f'] = 'JSON'
    else:
        params['f'] = 'CSV'

    response = requests.get(url, params=params)
    if response.status_code == 200:
        # Handle stacked metric response
        if url in METRICS_STACKED and key_to_extract:
            data = response.json()
            # Create column name combining metric name and the specific key
            column_name = f"{metric_base_name}_{key_to_extract}"
            
            # Extract timestamps and the specific key we want
            timestamps = [entry['t'] for entry in data]
            values = [entry['o'][key_to_extract] for entry in data]
            
            df = pd.DataFrame({
                't': timestamps,
                column_name: values
            })
        else:
            # Handle regular metric response
            df = pd.read_csv(io.StringIO(response.text))
            # Keep original column names from CSV
            if len(df.columns) != 2:
                print(f"Unexpected number of columns in response from {url}: {df.columns}")
                return None
            df.columns = ['t', metric_base_name]
        
        df['t'] = pd.to_datetime(df['t'], unit='s')
        # Convert the metric column to numeric, excluding the timestamp column
        for col in df.columns:
            if col != 't':
                df[col] = pd.to_numeric(df[col], errors='coerce')
        return df
    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

# Fetch and merge data
print("Fetching price data...")
price_df = fetch_glassnode_data(PRICE_URL)
all_dfs = [price_df]

# Fetch regular metrics
print("Fetching regular metrics...")
for metric_url in METRICS:
    print(f"Fetching {metric_url.split('/')[-1]}...")
    metric_df = fetch_glassnode_data(metric_url)
    if metric_df is not None:
        all_dfs.append(metric_df)

# Fetch stacked metrics
print("Fetching stacked metrics...")
for metric_url, keys in METRICS_STACKED.items():
    # Handle both single key (string) and multiple keys (list) cases
    if isinstance(keys, str):
        keys = [keys]  # Convert single key to list
    for key in keys:
        print(f"Fetching {metric_url.split('/')[-1]} - {key}...")
        metric_df = fetch_glassnode_data(metric_url, key_to_extract=key)
        if metric_df is not None:
            all_dfs.append(metric_df)

print("Merging all dataframes...")
merged_df = pd.concat(all_dfs, axis=1)
merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]
merged_df.set_index('t', inplace=True)

print("Final columns in merged_df:", merged_df.columns.tolist())

Fetching price data...
Fetching regular metrics...
Fetching us_spot_etf_flows_net...
Fetching stacked metrics...
Fetching us_spot_etf_balances_all - total...
Fetching us_spot_etf_balances_all - GBTC...
Fetching us_spot_etf_flows_all - total...
Fetching us_spot_etf_flows_all - GBTC...
Merging all dataframes...
Final columns in merged_df: ['price_usd_close', 'us_spot_etf_flows_net', 'us_spot_etf_balances_all_total', 'us_spot_etf_balances_all_GBTC', 'us_spot_etf_flows_all_total', 'us_spot_etf_flows_all_GBTC']


In [126]:
import yfinance as yf
from datetime import date, datetime, time, timedelta

# [Previous code remains the same until the final print statement]
print("Final columns in merged_df:", merged_df.columns.tolist())

def stock_ticker(ticker):
    # Download historical financial data
    response = yf.download(ticker, start="2024-01-17", end=datetime.combine(date.today(), time()))
    response['date'] = pd.to_datetime(response.index,utc=True)
    response = response.set_index('date',drop=False)
    response.index.name = ''
    return response

# Add volume data to merged_df
print("Fetching and adding ETF volume data...")

assets = [
    'GBTC','BTC','IBIT','FBTC','ARKB',
    'BITB','BTCO','HODL','BRRR',
    'EZBC','BTCW'
]

volume_df = pd.DataFrame(columns=['date'])
count = 0

for i in assets:
    try:
        etf = stock_ticker(i)
        vol_name = i+'_Volume'
        price_name = i+'_Price'
        
        if count == 0:
            volume_df['date'] = etf['date']
            volume_df['AGG_Volume'] = 0
            
        volume_df[price_name] = etf['Close']
        volume_df[vol_name] = etf['Volume']*etf['Close']
        volume_df[vol_name] = volume_df[vol_name].fillna(0)
        volume_df['AGG_Volume'] = volume_df['AGG_Volume'] + volume_df[vol_name]
        count += 1
    except Exception as e:
        print(f"Error processing {i}: {str(e)}")
        continue

# Convert merged_df index to UTC timezone to match volume_df
merged_df.index = pd.to_datetime(merged_df.index).tz_localize('UTC')

# Merge with existing dataframe
merged_df = pd.concat([merged_df, volume_df.set_index('date')], axis=1)
merged_df = merged_df.fillna(0)
print("Updated columns in merged_df:", merged_df.columns.tolist())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Final columns in merged_df: ['price_usd_close', 'us_spot_etf_flows_net', 'us_spot_etf_balances_all_total', 'us_spot_etf_balances_all_GBTC', 'us_spot_etf_flows_all_total', 'us_spot_etf_flows_all_GBTC']
Fetching and adding ETF volume data...
Updated columns in merged_df: ['price_usd_close', 'us_spot_etf_flows_net', 'us_spot_etf_balances_all_total', 'us_spot_etf_balances_all_GBTC', 'us_spot_etf_flows_all_total', 'us_spot_etf_flows_all_GBTC', 'AGG_Volume', 'GBTC_Price', 'GBTC_Volume', 'BTC_Price', 'BTC_Volume', 'IBIT_Price', 'IBIT_Volume', 'FBTC_Price', 'FBTC_Volume', 'ARKB_Price', 'ARKB_Volume', 'BITB_Price', 'BITB_Volume', 'BTCO_Price', 'BTCO_Volume', 'HODL_Price', 'HODL_Volume', 'BRRR_Price', 'BRRR_Volume', 'EZBC_Price', 'EZBC_Volume', 'BTCW_Price', 'BTCW_Volume']


In [127]:
volume_df

,date,AGG_Volume,GBTC_Price,GBTC_Volume,BTC_Price,BTC_Volume,IBIT_Price,IBIT_Volume,FBTC_Price,FBTC_Volume,...,BTCO_Price,BTCO_Volume,HODL_Price,HODL_Volume,BRRR_Price,BRRR_Volume,EZBC_Price,EZBC_Volume,BTCW_Price,BTCW_Volume
,,,,,,,,,,,,,,,,,,,,,
2024-01-17 00:00:00+00:00,2024-01-17 00:00:00+00:00,2.187420e+09,37.939999,1.398355e+09,NaN,0.000000e+00,24.410000,3.408734e+08,37.400002,2.959462e+08,...,42.810001,3.890573e+07,48.580002,6.402844e+06,12.160000,7.709440e+05,24.760000,4.830676e+06,45.360001,1.977696e+06
2024-01-18 00:00:00+00:00,2024-01-18 00:00:00+00:00,2.112808e+09,36.290001,1.060198e+09,NaN,0.000000e+00,23.340000,4.159935e+08,35.759998,3.673196e+08,...,40.849998,6.735756e+07,46.419998,1.024025e+07,11.590000,1.418616e+06,23.730000,7.470204e+06,43.500000,1.313700e+06
2024-01-19 00:00:00+00:00,2024-01-19 00:00:00+00:00,2.599274e+09,37.009998,1.378870e+09,NaN,0.000000e+00,23.799999,4.347189e+08,36.490002,5.038977e+08,...,41.564999,7.242285e+07,47.209999,9.456163e+06,11.820000,3.282414e+06,24.209999,3.972861e+06,44.360001,8.029160e+05
2024-01-22 00:00:00+00:00,2024-01-22 00:00:00+00:00,2.092623e+09,35.820000,1.079690e+09,NaN,0.000000e+00,22.950001,3.952472e+08,35.180000,4.095058e+08,...,40.200001,8.482200e+06,45.680000,6.084576e+06,11.410000,4.149817e+06,23.340000,5.015766e+06,42.799999,4.160160e+06
2024-01-23 00:00:00+00:00,2024-01-23 00:00:00+00:00,1.599548e+09,34.880001,7.632023e+08,NaN,0.000000e+00,22.320000,2.997955e+08,34.340000,3.580460e+08,...,39.169998,4.853163e+06,44.410000,7.625197e+06,11.120000,7.828480e+05,22.740000,3.595194e+06,41.650002,1.049580e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-22 00:00:00+00:00,2024-11-22 00:00:00+00:00,5.424985e+09,78.870003,4.523116e+08,44.049999,1.028700e+08,56.490002,3.729493e+09,86.730003,6.252453e+08,...,99.250000,2.537822e+07,112.379997,5.177346e+07,28.080000,5.972616e+06,57.599998,2.360448e+07,105.334999,1.089164e+07
2024-11-25 00:00:00+00:00,2024-11-25 00:00:00+00:00,5.566146e+09,75.419998,4.086934e+08,42.160000,9.077891e+07,54.020000,3.748810e+09,82.970001,6.640255e+08,...,94.870003,3.094659e+07,107.300003,4.566688e+07,26.850000,9.639150e+06,54.980000,1.725822e+07,100.480003,8.550848e+06
2024-11-26 00:00:00+00:00,2024-11-26 00:00:00+00:00,4.941052e+09,72.169998,3.922151e+08,40.320000,6.841901e+07,51.700001,3.485785e+09,79.449997,6.109546e+08,...,90.839996,3.041323e+07,102.889999,2.177152e+07,25.730000,7.214692e+06,52.740002,8.559702e+06,96.430000,6.345094e+06


In [128]:
merged_df

,price_usd_close,us_spot_etf_flows_net,us_spot_etf_balances_all_total,us_spot_etf_balances_all_GBTC,us_spot_etf_flows_all_total,us_spot_etf_flows_all_GBTC,AGG_Volume,GBTC_Price,GBTC_Volume,BTC_Price,...,BTCO_Price,BTCO_Volume,HODL_Price,HODL_Volume,BRRR_Price,BRRR_Volume,EZBC_Price,EZBC_Volume,BTCW_Price,BTCW_Volume
2024-01-17 00:00:00+00:00,42730.926233,3.747317e+08,2.779013e+10,2.525151e+10,3.747317e+08,-5.876709e+08,2.187420e+09,37.939999,1.398355e+09,0.00,...,42.810001,3.890573e+07,48.580002,6.402844e+06,12.160000,7.709440e+05,24.760000,4.830676e+06,45.360001,1.977696e+06
2024-01-18 00:00:00+00:00,41261.242495,1.727958e+07,2.682575e+10,2.370797e+10,1.727958e+07,-4.447305e+08,2.112808e+09,36.290001,1.060198e+09,0.00,...,40.849998,6.735756e+07,46.419998,1.024025e+07,11.590000,1.418616e+06,23.730000,7.470204e+06,43.500000,1.313700e+06
2024-01-19 00:00:00+00:00,41609.562508,-4.745801e+08,2.704494e+10,2.353655e+10,-4.745801e+08,-6.726037e+08,2.599274e+09,37.009998,1.378870e+09,0.00,...,41.564999,7.242285e+07,47.209999,9.456163e+06,11.820000,3.282414e+06,24.209999,3.972861e+06,44.360001,8.029160e+05
2024-01-20 00:00:00+00:00,41648.718646,0.000000e+00,2.704494e+10,2.353655e+10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
2024-01-21 00:00:00+00:00,41535.759163,0.000000e+00,2.704494e+10,2.353655e+10,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-27 00:00:00+00:00,95981.580554,1.486635e+07,1.028637e+11,2.082636e+10,1.486635e+07,-3.922503e+07,4.633157e+09,76.820000,3.022022e+08,42.93,...,96.589996,3.292753e+07,109.360001,1.699454e+07,27.370001,8.079624e+06,56.080002,1.185531e+07,102.540001,1.303283e+07
2024-11-28 00:00:00+00:00,95651.196450,5.449006e+07,1.043016e+11,2.082636e+10,5.449006e+07,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00
2024-11-29 00:00:00+00:00,97447.132368,1.658478e+08,1.048800e+11,2.099483e+10,1.658478e+08,-1.721195e+06,2.483001e+09,77.089996,2.287337e+08,43.09,...,97.010002,8.721199e+06,109.779999,1.429336e+07,27.480000,6.729852e+06,56.320000,6.172672e+06,103.029999,1.135391e+07
2024-11-30 00:00:00+00:00,96514.052710,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.00,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00


In [129]:
def calculate_smoothed_bands_v0(df, column = 'us_spot_etf_flows_net', smoothing_fun='sma', smoothing_window=1, range_method='maxmin', range_window=90):
  # Calculate product of the two columns
  product = df[column].rolling(window=7).sum()
  # Calculate smoothed version based on method using the product
  if smoothing_fun == 'sma':
      smoothed_raw = product.rolling(window=smoothing_window).mean()
  elif smoothing_fun == 'ema':
      smoothed_raw = product.ewm(span=smoothing_window).mean()
  elif smoothing_fun == 'med':
      smoothed_raw = product.rolling(window=smoothing_window).median()
  else:
      raise ValueError("Invalid smoothing method. Choose 'sma', 'ema', or 'med'.")
  
  # Calculate bands based on smoothed data
  if range_method == 'maxmin':
      upper_band = smoothed_raw.rolling(window=range_window).max()
      lower_band = smoothed_raw.rolling(window=range_window).min()
  
  elif range_method == 'zscore':
      mean = smoothed_raw.rolling(window=range_window).mean()
      std = smoothed_raw.rolling(window=range_window).std()
      upper_band = mean + std
      lower_band = mean - std
  
  elif range_method == 'percentile':
      upper_band = smoothed_raw.rolling(window=range_window).quantile(0.95)
      lower_band = smoothed_raw.rolling(window=range_window).quantile(0.05)
  
  elif range_method == 'meandev':
      mean = smoothed_raw.rolling(window=range_window).mean()
      mean_deviation = (smoothed_raw - mean).abs().rolling(window=range_window).mean()
      upper_band = mean + mean_deviation
      lower_band = mean - mean_deviation
  
  else:
      raise ValueError("Invalid range method. Choose 'maxmin', 'zscore', 'percentile', or 'meandev'.")
  
  return smoothed_raw, upper_band, lower_band

In [130]:
def calculate_smoothed_bands_v1(df, columns=['us_spot_etf_balances_all_total','us_spot_etf_balances_all_GBTC','us_spot_etf_flow_all_total','us_spot_etf_flow_all_GBTC'], smoothing_fun='sma', smoothing_window=1,range_method='maxmin', range_window=90):
    
    ref_date = "2024-01-17"
    start_date = "2024-01-27"
    
    # Calculate the product according to the formula
    product = ((df[columns[0]] - df[columns[1]] - (df[columns[0]] - df[columns[1]]).loc[ref_date]) / 
          (df[columns[2]].cumsum() - df[columns[3]].cumsum()))
    
    # Subset the data from the specified date
    product = product[product.index >= start_date]
    
    # Calculate smoothed version based on method using the product
    if smoothing_fun == 'sma':
        smoothed_raw = product.rolling(window=smoothing_window).mean()
    elif smoothing_fun == 'ema':
        smoothed_raw = product.ewm(span=smoothing_window).mean()
    elif smoothing_fun == 'med':
        smoothed_raw = product.rolling(window=smoothing_window).median()
    else:
        raise ValueError("Invalid smoothing method. Choose 'sma', 'ema', or 'med'.")
    
    # Calculate bands based on smoothed data
    if range_method == 'maxmin':
        upper_band = smoothed_raw.rolling(window=range_window).max()
        lower_band = smoothed_raw.rolling(window=range_window).min()
    
    elif range_method == 'zscore':
        mean = smoothed_raw.rolling(window=range_window).mean()
        std = smoothed_raw.rolling(window=range_window).std()
        upper_band = mean + std
        lower_band = mean - std
    
    elif range_method == 'percentile':
        upper_band = smoothed_raw.rolling(window=range_window).quantile(0.95)
        lower_band = smoothed_raw.rolling(window=range_window).quantile(0.05)
    
    elif range_method == 'meandev':
        mean = smoothed_raw.rolling(window=range_window).mean()
        mean_deviation = (smoothed_raw - mean).abs().rolling(window=range_window).mean()
        upper_band = mean + mean_deviation
        lower_band = mean - mean_deviation
    
    else:
        raise ValueError("Invalid range method. Choose 'maxmin', 'zscore', 'percentile', or 'meandev'.")
    
    return smoothed_raw, upper_band, lower_band

In [131]:
merged_df['etfFlow_sma_meandev'], merged_df['etfFlow_upper'], merged_df['etfFlow_lower']= calculate_smoothed_bands_v0(merged_df, column='us_spot_etf_flows_net',smoothing_fun='sma', smoothing_window=1, range_method='meandev', range_window=30)

merged_df['etfmvrv_sma_meandev'], merged_df['etfmvrv_upper'], merged_df['etfmvrv_lower']= calculate_smoothed_bands_v1(merged_df, columns=['us_spot_etf_balances_all_total','us_spot_etf_balances_all_GBTC','us_spot_etf_flows_all_total','us_spot_etf_flows_all_GBTC'],smoothing_fun='sma', smoothing_window=1, range_method='meandev', range_window=30)

merged_df['etfvol_sma_meandev'], merged_df['etfvol_upper'], merged_df['etfvol_lower']= calculate_smoothed_bands_v0(merged_df, column='AGG_Volume',smoothing_fun='sma', smoothing_window=1, range_method='meandev', range_window=30)

In [132]:
def create_indicator_chart(merged_df, indicator_column, chart_title, bands="on", upper_lower=["fee_upper", "fee_lower"]):
   # Filter data for the last year
   one_year_ago = pd.Timestamp.now(tz='UTC') - timedelta(days=730)
   merged_df_last_year = merged_df[merged_df.index > one_year_ago]

   # Define colors
   GREY_COLOR = 'rgba(128, 128, 128, 0.7)'  # Semi-transparent grey
   BLUE_COLOR = 'rgb(0, 0, 255)'  # Blue
   GREEN_COLOR = 'rgb(0, 255, 0)'  # Green for upper band
   RED_COLOR = 'rgb(255, 0, 0)'    # Red for lower band

   # Create the visualization
   fig = make_subplots(specs=[[{"secondary_y": True}]])

   # Add price trace
   fig.add_trace(
       go.Scatter(x=merged_df_last_year.index, y=merged_df_last_year['price_usd_close'], 
                 name="Price USD", line=dict(color=GREY_COLOR, width=2), mode='lines'),
       secondary_y=False,
   )

   # Add indicator trace
   indicator = merged_df_last_year[indicator_column]
   fig.add_trace(
       go.Scatter(
           x=merged_df_last_year.index,
           y=indicator,
           name=indicator_column,
           line=dict(color=BLUE_COLOR, width=2),
           mode='lines'
       ),
       secondary_y=True,
   )

   # Add band traces if enabled
   if bands == "on":
       # Simply use the provided column names directly
       fig.add_trace(
           go.Scatter(
               x=merged_df_last_year.index,
               y=merged_df_last_year[upper_lower[0]], 
               name="Upper Band",
               line=dict(color=GREEN_COLOR, width=2, dash='dash'),
               mode='lines'
           ),
           secondary_y=True,
       )
       fig.add_trace(
           go.Scatter(
               x=merged_df_last_year.index,
               y=merged_df_last_year[upper_lower[1]], 
               name="Lower Band",
               line=dict(color=RED_COLOR, width=2, dash='dash'),
               mode='lines'
           ),
           secondary_y=True,
       )

   # Add vertical lines for every two months (Jan, Mar, May, Jul, Sep, Nov)
   for month in [1, 4, 7, 10]:
       for year in range(merged_df_last_year.index[0].year, merged_df_last_year.index[-1].year + 1):
           # Make the date timezone-aware
           date = pd.Timestamp(year=year, month=month, day=1, tz='UTC')
           if merged_df_last_year.index[0] <= date <= merged_df_last_year.index[-1]:
               fig.add_vline(x=date, line_dash="dash", line_color=GREY_COLOR, line_width=0.75, opacity=0.7)

   # Function to format number to K, M, B
   def format_number(num):
       if abs(num) >= 1e9:
           return f"{num/1e9:.2f}B"
       elif abs(num) >= 1e6:
           return f"{num/1e6:.2f}M"
       elif abs(num) >= 1e3:
           return f"{num/1e3:.2f}K"
       else:
           return f"{num:.2f}"
   
   last_value = indicator.dropna().iloc[-1] if not indicator.dropna().empty else np.nan
   formatted_value = format_number(last_value)

   # Add annotation for the last value
   fig.add_annotation(
       x=0.95,  
       y=last_value*0.88,  
       xref="paper",
       yref="y2", 
       text=formatted_value,
       showarrow=False,
       font=dict(size=18, color=BLUE_COLOR),
       align="left",
       xanchor="left",
       yanchor="middle",
   )

   # Update layout
   fig.update_layout(
       title={
           'text': chart_title,
           'font': {'color': 'black', 'size': 18, 'weight': 'bold'}
       },
       showlegend=False,  
       hovermode="x unified",
       plot_bgcolor='white',
       paper_bgcolor='white',
       font={'color': 'black', 'size': 14},
       width=1000,  
       height=450,  
   )

   # Update axes
   fig.update_xaxes(
       showgrid=False, 
       tickfont={'color': 'black', 'size': 14},
       zeroline=False,
       title_text=''  
   )
   fig.update_yaxes(
       showgrid=False, 
       secondary_y=False, 
       tickfont={'color': GREY_COLOR, 'size': 14},
       zeroline=False,
       showline=True,
       linecolor=GREY_COLOR,
       ticks='outside',
       tickcolor=GREY_COLOR,
       title_text='',
       title_font=dict(size=18)
   )
   fig.update_yaxes(
       showgrid=False, 
       secondary_y=True, 
       tickfont={'color': GREY_COLOR, 'size': 14},
       zeroline=False,
       showline=True,
       linecolor=GREY_COLOR,
       ticks='outside',
       side='right',
       tickcolor=GREY_COLOR,
       title_text='',
       title_font=dict(size=18),
       autorange=True
   )

   return fig

In [133]:
# Create charts for each indicator
indicators = [
    ('etfFlow_sma_meandev',"Bitcoin: US Spot ETF Netflow", "on", ["etfFlow_upper", "etfFlow_lower"]),
    ('etfmvrv_sma_meandev',"Bitcoin: US Spot ETF MVRV (Exc. GBTC)", "on", ["etfmvrv_upper", "etfmvrv_lower"]),
    ('etfvol_sma_meandev',"Bitcoin: US Spot ETF Trade Volume (Weekly)", "on", ["etfvol_upper", "etfvol_lower"])
]

for indicator, title,  bands, upper_lower in indicators:
    fig = create_indicator_chart(merged_df, indicator, title, bands, upper_lower)
    
    # Show the plot
    fig.show()
    
    # Optionally, save the plot as an HTML file
    pio.write_html(fig, file=f'bitcoin_analysis_{indicator}_two_year.html')

print("All charts have been displayed and saved as separate HTML files.")

All charts have been displayed and saved as separate HTML files.


In [134]:
merged_df.to_csv('bitcoin_analysis_data_spotETF2.csv', index=True)

In [135]:
merged_df.tail()

,price_usd_close,us_spot_etf_flows_net,us_spot_etf_balances_all_total,us_spot_etf_balances_all_GBTC,us_spot_etf_flows_all_total,us_spot_etf_flows_all_GBTC,AGG_Volume,GBTC_Price,GBTC_Volume,BTC_Price,...,BTCW_Volume,etfFlow_sma_meandev,etfFlow_upper,etfFlow_lower,etfmvrv_sma_meandev,etfmvrv_upper,etfmvrv_lower,etfvol_sma_meandev,etfvol_upper,etfvol_lower
2024-11-27 00:00:00+00:00,95981.580554,1.486635e+07,1.028637e+11,2.082636e+10,1.486635e+07,-3.922503e+07,4.633157e+09,76.820000,3.022022e+08,42.93,...,1.303283e+07,9.926567e+08,3.019938e+09,1.116898e+09,1.582710,1.595824,1.242486,2.770001e+10,3.014093e+10,1.428609e+10
2024-11-28 00:00:00+00:00,95651.196450,5.449006e+07,1.043016e+11,2.082636e+10,5.449006e+07,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.00,...,0.000000e+00,3.393304e+07,2.967136e+09,1.023462e+09,1.609590,1.610104,1.252545,2.056534e+10,3.027087e+10,1.461489e+10
2024-11-29 00:00:00+00:00,97447.132368,1.658478e+08,1.048800e+11,2.099483e+10,1.658478e+08,-1.721195e+06,2.483001e+09,77.089996,2.287337e+08,43.09,...,1.135391e+07,-3.378057e+08,2.865954e+09,9.068630e+08,1.612369,1.626191,1.262923,1.762336e+10,3.038476e+10,1.473634e+10
2024-11-30 00:00:00+00:00,96514.052710,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.00,...,0.000000e+00,-3.378057e+08,2.779040e+09,7.914420e+08,-0.050318,1.630853,1.174898,1.762336e+10,3.040445e+10,1.484813e+10
2024-12-01 00:00:00+00:00,97311.455238,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.00,...,0.000000e+00,-3.378057e+08,2.722955e+09,6.770472e+08,-0.050318,1.635316,1.088098,1.762336e+10,3.042054e+10,1.495119e+10


In [136]:
import plotly.graph_objects as go
from datetime import datetime, timedelta

# Change the grey color to a darker shade
GREY_COLOR = 'rgba(70, 70, 70, 0.9)'  # Darker, more opaque grey

def normalize_value(value, min_val, max_val):
    range_val = max_val - min_val
    if range_val == 0:
        return 0
    normalized = (value - min_val) / range_val * 200 - 100
    return normalized

def format_number(number):
    """Format number with k, M, B suffixes"""
    abs_number = abs(number)
    if abs_number >= 1e9:
        return f"{number/1e9:.2f}B"
    elif abs_number >= 1e6:
        return f"{number/1e6:.2f}M"
    elif abs_number >= 1e3:
        return f"{number/1e3:.2f}k"
    else:
        return f"{number:.2f}"

def create_dot_plot(df, metrics_titles, days=90):
    start_date = df.index[-1] - timedelta(days=days)
    df_filtered = df[df.index >= start_date]
    
    fig = go.Figure()
    y_values = []
    
    for metric, title in metrics_titles.items():
        min_val = df_filtered[metric].min()
        max_val = df_filtered[metric].max()
        current_val = df_filtered[metric].iloc[-1]
        
        # Normalize values
        norm_min = -100
        norm_max = 100
        norm_current = normalize_value(current_val, min_val, max_val)
        
        y_values.append(title)
        
        # Add grey line (darker)
        fig.add_trace(go.Scatter(
            x=[norm_min, norm_max], y=[title, title], mode='lines',
            line=dict(color=GREY_COLOR, width=3),  # Using darker grey
            showlegend=False
        ))
        
        # Add min value (red filled dot)
        fig.add_trace(go.Scatter(
            x=[norm_min], y=[title], mode='markers+text',
            marker=dict(color='red', size=16, symbol='circle'),
            text=[format_number(min_val)],
            textposition="bottom center",
            textfont=dict(color=GREY_COLOR, size=16),  # Using darker grey
            name=f'{title} Min'
        ))
        
        # Add max value (green filled dot)
        fig.add_trace(go.Scatter(
            x=[norm_max], y=[title], mode='markers+text',
            marker=dict(color='green', size=16, symbol='circle'),
            text=[format_number(max_val)],
            textposition="bottom center",
            textfont=dict(color=GREY_COLOR, size=16),  # Using darker grey
            name=f'{title} Max'
        ))
        
        # Add current value (grey halo dot)
        fig.add_trace(go.Scatter(
            x=[norm_current], y=[title], mode='markers+text',
            marker=dict(
                color='white',
                size=16,
                line=dict(color=GREY_COLOR, width=3),  # Using darker grey
                symbol='circle'
            ),
            text=[format_number(current_val)],
            textposition="top center",
            textfont=dict(color=GREY_COLOR, size=16),  # Using darker grey
            name=f'{title} Current'
        ))
    
    # Add vertical dashed lines at -50%, 0%, and 50%
    for value in [-50, 0, 50]:
        fig.add_shape(
            type="line",
            x0=value, x1=value, y0=0, y1=1,
            yref="paper",
            xref="x",
            line=dict(color=GREY_COLOR, width=1, dash="dash")  # Using darker grey
        )
    
    fig.update_layout(
        title={
            'text': f'Metric Values Over Last {days} Days',
            'font': {'color': GREY_COLOR, 'size': 20},  # Using darker grey
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_title={
            'text': 'Normalized Value (%)',
            'font': {'color': GREY_COLOR, 'size': 16}  # Using darker grey
        },
        yaxis=dict(
            tickmode='array',
            tickvals=list(range(len(y_values))),
            ticktext=y_values,
            tickfont={'color': GREY_COLOR, 'size': 16},  # Using darker grey
            side='left',
            title_standoff=35
        ),
        height=400 + (len(metrics_titles) * 30),
        width=1200,
        showlegend=False,
        plot_bgcolor='white',
        paper_bgcolor='white',
        margin=dict(l=100, r=100, t=50, b=20),
        font=dict(color=GREY_COLOR, size=14)  # Using darker grey
    )
    
    fig.update_xaxes(
        showgrid=False, 
        range=[-110, 110],
        tickvals=[-100, -50, 0, 50, 100],
        ticktext=['-100%<br>MIN-90D', '-50%', '0%', '50%', '100%<br>MAX-90D'],
        tickfont={'color': GREY_COLOR, 'size': 16},  # Using darker grey
        showline=False,
        zeroline=False
    )
    fig.update_yaxes(showgrid=False)
    
    return fig

In [137]:
# Manually define metrics and their titles
metrics_titles = {

    'etfFlow_sma_meandev': "US Spot ETF Netflow",
    'etfmvrv_sma_meandev': "US Spot ETF MVRV (Exc. GBTC)",
    'etfvol_sma_meandev': "US Spot ETF Volume (Weekly)"
       # Add more metrics and titles as needed
}

# Create the dot plot
fig = create_dot_plot(merged_df, metrics_titles)

# Show the plot
fig.show()

# Optionally, save the plot as an HTML file
import plotly.io as pio
pio.write_html(fig, file='normalized_spotETF2_raw_metrics_dot_plot.html')